The FWS Threatened and Endangered Species (TESS) system is part of their underlying data source. TESS has a set of web services that are a little rudimentary but functional. In this notebook, we retrieve information from TESS, first for all of the species we "ECOS-scraped" that have ITIS TSN numbers declared (cached to the cache/workplan_species.json file), and then we look for any other species in the list where we did not get a TSN.

The main bispy function here is the search function in the Tess class of the tess module. It takes either an ITIS TSN or scientific name and runs a search. The TESS service returns XML, but the function transforms and tweaks slightly to return a dictionary (JSON) structure.

In [1]:
import json
import bispy
from IPython.display import display
from joblib import Parallel, delayed

tess = bispy.tess.Tess()
bis_utils = bispy.bis.Utils()

In [2]:
# Open up the cached workplan species
with open("cache/workplan_species.json", "r") as f:
    workplan_species = json.loads(f.read())

In [3]:
# Prepare two lists - one of the TSNs we already know about from ECOS scraping and the other of the remaining scientific names
tsn_list = [r["ITIS TSN"] for r in workplan_species if r["ITIS TSN"] is not None]
names_without_tsns = [r["Lookup Name"] for r in workplan_species if r["ITIS TSN"] is None]

In [5]:
%%time
# Use joblib to run multiple requests for TESS documents in parallel via known ITIS TSNs
tess_cache_from_tsn = Parallel(n_jobs=8)(delayed(tess.search)(tsn) for tsn in tsn_list)

CPU times: user 734 ms, sys: 111 ms, total: 845 ms
Wall time: 24.3 s


In [6]:
%%time
# Use joblib to run multiple requests for TESS documents in parallel via scientific names
tess_cache_from_names = Parallel(n_jobs=8)(delayed(tess.search)(name) for name in names_without_tsns)

CPU times: user 239 ms, sys: 16.6 ms, total: 256 ms
Wall time: 9.64 s


In [17]:
# If any new TSNs were found via name search, update those back to the workplan species JSON
updated_workplan_species = list()
for record in [t for t in tess_cache_from_names if 'TESS Species' in t.keys() and int(t['TESS Species']['SPECIES_DETAIL']['TSN']) > 0]:
    workplan_record = next((d for d in workplan_species if d["Lookup Name"] == record['TESS Species']['SPECIES_DETAIL']['SCINAME']), None)
    workplan_record["ITIS TSN"] = record["TESS Species"]["SPECIES_DETAIL"]["TSN"]
    updated_workplan_species.append(workplan_record)
    updated_species_names = [s["Scientific Name"] for s in updated_workplan_species]

if len(updated_workplan_species) > 0:
    new_workplan_species = list(filter(lambda i: i['Scientific Name'] not in updated_species_names, workplan_species))
    new_workplan_species.extend(updated_workplan_species)
    display(bis_utils.doc_cache("cache/workplan_species.json", new_workplan_species))
    

{'Doc Cache File': 'cache/workplan_species.json',
 'Document Number 133': {'Bin': None,
  'ECOS Link': 'https://ecos.fws.gov/ecp/species/4851',
  'Guild': 'Crustaceans',
  'ITIS TSN': '97369',
  'Lead FWS Regional Office': 'Region 4 - Southeast',
  'Lookup Name': 'Cambarus georgiae',
  'Proposed FWS Decision Timeframe (Fiscal Year)': 2023,
  'Range': 'GA, NC',
  'Scientific Name': 'Cambarus georgiae',
  'Species Name (Common)': 'Little Tennessee crayfish'},
 'Number of Documents in Cache': 363}

In [18]:
# Cache the array of retrieved documents and return/display a random sample for verification
display(bis_utils.doc_cache("cache/tess.json", tess_cache_from_tsn))

{'Doc Cache File': 'cache/tess.json',
 'Document Number 249': {'Processing Metadata': {'Date Processed': '2019-07-03T21:14:36.897605',
   'Search URL': 'https://ecos.fws.gov/ecp0/TessQuery?request=query&xquery=/SPECIES_DETAIL[TSN=568412]',
   'Status': 'Successful Match'},
  'TESS Species': {'SPECIES_DETAIL': {'COMNAME': 'Amargosa tryonia',
    'COUNTRY': '1',
    'DPS': '0',
    'ENTITY_ID': '7000',
    'FAMILY': 'Hydrobiidae',
    'INVNAME': 'Tryonia, Amargosa',
    'LEAD_AGENCY': '1',
    'LEAD_REGION': '8',
    'POP_ABBREV': 'Wherever found',
    'POP_DESC': 'Wherever found',
    'REFUGE_OCCURRENCE': None,
    'SCINAME': 'Tryonia variegata',
    'SPCODE': 'G011',
    'STATUS': 'UR',
    'STATUS_TEXT': 'Under Review in the Candidate or Petition Process',
    'TSN': '568412',
    'VIPCODE': 'I01'}}},
 'Number of Documents in Cache': 266}